In [39]:
import random
import time
from IPython.display import clear_output

In [40]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
def print_rules() -> None:
    """
    Prints hangman rules.
    """
    start_man = """
         ______
        |      |
        |
        |
        |
        |
      __|________

    """

    print(f'Welcome to Hangman!')
    print(f'Rules: Guess the word before the hangman gains all its limbs.')
    print(start_man)

In [42]:
def pick_word() -> str:
    """
    Picks a random word from a word bank.

    Returns:
        (str):      Returns an uppercase word in a string.
    """
    folder_path = '/content/gdrive/MyDrive/Colab Notebooks/text_files/'

    with open(folder_path + "english-common-words.txt", "r") as file:
        lines = file.readlines()

    word = random.choice(lines)     # randomly chooses a line from the file (each line is a word)

    return word.strip().upper()     # returns the word in all uppercase letters

In [43]:
def letter_check(word: str, user_letter: str, guessed_word: set, incorrect_letters: set) -> int:
    """
    Checks if the user guessed letter is a valid character, if it has already been guessed, or if it is in the random word.

    Args:
        word              (str):        A randomly chosen 5 letter word from a file containing words.
        user_letter       (str):        A string containing a single alphabetic letter.
        guessed_word      (set):        A set of all letters that are in the chosen word.
        incorrect_letters (set):        A set of all letters that are not in the chosen word.

    Returns:
        (int):       Returns an integer value depending on whether the user inputted letter is valid, already guessed, or in the word.

    Examples:
        >>> letter_check("HELLO", "L", set(), {"Z", "P"})
            3

        >>> letter_check("GOODBYE", "H", {"E", "O"}, {"R", "S", "T"})
            4

        >>> letter_check("SUPER", "UU", {"R"}, {"L"})
            1

        >>> letter_check("HIPPO", "P", {"P", "O"}, set())
            2
    """
    if not user_letter.isalpha() or len(user_letter) != 1:      # checks if letter is not in alphabet or if it is longer than 1
        return 1

    elif user_letter in guessed_word or user_letter in incorrect_letters:       # checks if letter has already been guessed
        return 2

    elif user_letter in word:               # checks if letter is in the word
        guessed_word.add(user_letter)       # adds letter to set of correctly guessed letters
        return 3

    else:
        incorrect_letters.add(user_letter)     # adds letter to set of incorrectly guessed letters
        return 4

In [44]:
def print_word_progress(word: str, user_letter: str, current_word: list) -> list:
    """
    Takes the current status of the chosen word and updates it with newly guessed letters in the right positions.

    Args:
        word              (str):        A randomly chosen 5 letter word from a file containing words.
        user_letter       (str):        A string containing a single alphabetic letter.
        current_word     (list):        A list containing the current status of the word. Elements are letters that are positioned as they would
                                        be in the word.

    Returns:
        current_word     (list):        A list containing the current status of the word. Elements are letters that are positioned as they would
                                        be in the word.

    Examples:
        >>> print_word_progress("HAPPY", "P", ["*", "*", "*", "*", "*"])
            ["*", "*", "P", "P", "*"]

        >>> print_word_progress("WONDER", "D", ["W", "*", "*", "*", "E", "R"])
            ["W", "*", "*", "D", "E", "R"]

        >>> print_word_progress("SUPER", "A", ["S", "*", "*", "E", "*"])
            ["S", "*", "*", "E", "*"]
    """
    for index in range(len(word)):              # loops through word
        if word[index] == user_letter:              # checks whether word at index is the same as the user's guessed letter
            current_word[index] = user_letter       # if it is the same, replaces a "*" with the letter at that index

    return current_word

In [45]:
def list_to_string(current_word: list) -> str:
    """
    Converts a list in which each element represents a letter of a word into a string.

    Args:
        current_word     (list):        A list containing the current status of the word. Elements are letters that are positioned as they would
                                        be in the word.

    Returns:
                          (str):        A string containing made up of the elements in the inputted list.
    Examples:
        >>> list_to_string(["H", "E", "L", "L", "O"])
            "HELLO"

        >>> list_to_string(["G", "O", "O", "D", "B", "Y", "E"])
            "GOODBYE"
    """
    word_string = ""

    for element in current_word:        # takes each element and adds them to a string to get the word in a string, not a list
        word_string += element

    return word_string

In [46]:
def print_hangman_pic(graphics_index: int) -> str:
    """
    Returns a depiction of a hangman representing different stages in the game.

    Args:
        graphics_index (int):        An integer 0-6 that is used to index a list of string graphics.

    Returns:
                       (str):        A multi-line string that represents a hangman drawing.
    """
    start_man = """
         ______
        |      |
        |
        |
        |
        |
      __|________ """

    first_man = """
         ______
        |      |
        |      O
        |
        |
        |
      __|________ """

    second_man = """
         ______
        |      |
        |      O
        |      |
        |
        |
      __|________ """

    third_man = """
         ______
        |      |
        |      O
        |      |
        |      |
        |
      __|________ """

    forth_man = """
         ______
        |      |
        |      O
        |    __|
        |      |
        |
      __|________ """

    fifth_man = """
         ______
        |      |
        |      O
        |    __|__
        |      |
        |
      __|________ """

    sixth_man = """
         ______
        |      |
        |      O
        |    __|__
        |      |
        |     /
      __|________ """

    end_man = """
         ______
        |      |
        |      O
        |    __|__
        |      |
        |     / \\
      __|________ """

    graphics_list = [start_man, first_man, second_man, third_man, forth_man, fifth_man, sixth_man, end_man]

    return graphics_list[graphics_index]

In [47]:
def play_hangman() -> None:
    """
    Prints whether or not the user inputted letter is valid, already guessed, and in or not in the word. Prints visuals for the hangman, as well
    as the word's progress throughout the game.
    """
    word = pick_word()          # calls for the random word to be picked
    guessed_word = set()        # this is the set of all correctly guessed letters
    incorrect_letters = set()      # this is the set of all incorrectly guessed letters

    current_word = ["*"] * len(word)    # current_word starts off as stars and gets updated with letters as the game progresses
    graphics_index = 0

    print_rules()
    print(f'WORD: {"*" * len(word)}\n')

    while len(guessed_word) < len(set(word)):       # keeps going as long as all the letters in the word have not been guessed

        clear_output(wait = True)       # clears the display

        time.sleep(.25)         # time delay

        user_letter = input("Guess a letter: ").strip().upper()
        word_result = letter_check(word, user_letter, guessed_word, incorrect_letters)

        if word_result == 1:
            print(f'\n{user_letter} is not accepted. Please enter a single alphabetic letter.\n')

            print(f'{print_hangman_pic(graphics_index)} \n')        # here we are printing a new picture where the hangman has gained a limb
            print(f'WORD: {list_to_string(print_word_progress(word, user_letter, current_word))}\n')
            print(f'Word does not contain: {list(incorrect_letters)}\n')

        elif word_result == 2:
            print(f'\n{user_letter} has already been guessed.\n')

            print(f'{print_hangman_pic(graphics_index)} \n')
            print(f'WORD: {list_to_string(print_word_progress(word, user_letter, current_word))}\n')
            print(f'Word does not contain: {list(incorrect_letters)}\n')

        elif word_result == 3:
            print(f'\n{user_letter} is in word!')

            print(f'{print_hangman_pic(graphics_index)} \n')
            print(f'WORD: {list_to_string(print_word_progress(word, user_letter, current_word))}\n')
            print(f'Word does not contain: {list(incorrect_letters)}\n')

        elif word_result == 4:
            print(f'\n{user_letter} is not in word.')
            graphics_index += 1             # this adds a limb to the hangman

            if graphics_index == 7:                     # if the index reaches 7, there are no more limbs to add -> hangman dies
                print(f'{print_hangman_pic(7)} \n')
                print(f'Your hangman died!\n\n WORD: {word}\n GAME OVER.')

                return

            else:
                print(f'{print_hangman_pic(graphics_index)} \n')
                print(f'WORD: {list_to_string(print_word_progress(word, user_letter, current_word))}\n')
                print(f'Word does not contain: {list(incorrect_letters)}\n')

    print(f'You guessed it! The word is {word}.')

In [ ]:
play_hangman()

Welcome to Hangman!
Rules: Guess the word before the hangman gains all its limbs.

         ______
        |      |
        |
        |
        |
        |
      __|________

    
WORD: ***

